In [4]:
import onnxruntime as ort
import numpy as np
import cv2

In [5]:
def preprocess_image(image_path, input_size):
    original_image = cv2.imread(image_path)
    orig_h, orig_w = original_image.shape[:2]
    img = cv2.resize(original_image, input_size)
    img = img.astype(np.float32) / 255.0
    img = np.transpose(img, (2, 0, 1))
    img = np.expand_dims(img, axis=0)
    return img, original_image, orig_h, orig_w

In [6]:
def save_image_with_boxes(outputs, min_conf, iou_threshold, x_scale, y_scale, original_image):
    raw_output = outputs[0][0]
    predictions = raw_output.transpose()

    boxes = []
    confidences = []

    for pred in predictions:
        confidence = pred[4]
        if confidence > min_conf:
            xc, yc, w, h = pred[0], pred[1], pred[2], pred[3]
            
            x = xc - w/2
            y = yc - h/2
            
            x_rescaled = x * x_scale
            y_rescaled = y * y_scale
            w_rescaled = w * x_scale
            h_rescaled = h * y_scale
            
            boxes.append([int(x_rescaled), int(y_rescaled), int(w_rescaled), int(h_rescaled)])
            confidences.append(float(confidence))

    indices = cv2.dnn.NMSBoxes(boxes, confidences, min_conf, iou_threshold)
    
    for i in indices.flatten(): 
        box = boxes[i]
        score = confidences[i]
        x, y, w, h = map(int, box)
        
        x2 = x + w
        y2 = y + h
        
        cv2.rectangle(original_image, (x, y), (x2, y2), (0, 255, 0), 2)
        
        label = f"Dechet: {score:.2f}"
        (w_text, h_text), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
        cv2.rectangle(original_image, (x, y - 20), (x + w_text, y), (0, 255, 0), -1)
        cv2.putText(original_image, label, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        
    cv2.imwrite("detections.jpg", original_image)


In [7]:
model_path = "/home/lois/Documents/mlops_project/models/yolov8n_taco_320/weights/best.onnx"
# image_path = "/home/lois/Downloads/PXL_20251121_202014620.jpg"
image_path = "/home/lois/Documents/mlops_project/data/Plastic-Waste-2/test/images/0008_jpg.rf.6af9a2ae7a3e4a4716ef1698a2dd0b22.jpg"

min_conf = 0.5
iou_threshold = 0.4
input_shape = (320, 320)

In [8]:
ort_session = ort.InferenceSession(model_path)
input_name = ort_session.get_inputs()[0].name

img, original_image, orig_h, orig_w = preprocess_image(image_path, input_shape)

x_scale = orig_w / input_shape[0]
y_scale = orig_h / input_shape[1]

outputs = ort_session.run(None, {input_name: img})

save_image_with_boxes(outputs, min_conf, iou_threshold, x_scale, y_scale, original_image)